In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
import matplotlib.patches as patches
import pickle
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.util import ngrams

from sklearn.utils import resample
from sklearn.decomposition import NMF
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [3]:
primaries = pd.read_csv('audience')

In [4]:
for idx, i in enumerate(primaries['tokenized']):
    primaries['tokenized'][idx] = i.replace('applaus', '')
    
for idx, i in enumerate(primaries['tokenized']):
    primaries['tokenized'][idx] = i.replace('cheer', '')

/Users/kevinpark/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/kevinpark/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [5]:
vectorizer = TfidfVectorizer(stop_words='english', max_features = 5000)
ss = SnowballStemmer(language='english')
def stemmer(arr):
    '''
    takes a corpus in an array and returns a simillar arr of stemmed words
    '''
    output = list()
    for text in arr:
        current = ""
        for word in text.split():
            current += ss.stem(word) + " "
        output.append(current)
    return output

def document_vectorizer(arr):
    step1 = stemmer(arr)
    step2 = vectorizer.fit_transform(np.array(step1))
    data = pd.DataFrame(step2.toarray(), columns = vectorizer.get_feature_names())
    return data

In [6]:
vector_pd = document_vectorizer(primaries['tokenized'])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(vector_pd, primaries['Label2'])


In [8]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred_class = nb.predict(X_test)
y_prob = nb.predict_proba(X_test)
print(accuracy_score(y_test, y_pred_class))
print(precision_score(y_test, y_pred_class))
print(recall_score(y_test, y_pred_class))

0.917437252311757
0.0
0.0


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
X = pd.concat([X_train, y_train], axis=1)

In [16]:
cheer = X[X.Label2 == 1]
not_cheer = X[X.Label2 == 0]

cheer_upsampled = resample(cheer,
                          replace=True, # sample with replacement
                          n_samples=len(not_cheer), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([not_cheer, cheer_upsampled])

y_train = upsampled.Label2
X_train = upsampled.drop('Label2', axis=1)

nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred_class = nb.predict(X_test)
y_prob = nb.predict_proba(X_test)
print('Multinomial NB')
print(accuracy_score(y_test, y_pred_class))
print(precision_score(y_test, y_pred_class))
print(recall_score(y_test, y_pred_class))

lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred_class = lr.predict(X_test)
lr_prob = lr.predict_proba(X_test)
print('Logistic Regression')
print(accuracy_score(y_test, lr_pred_class))
print(precision_score(y_test, lr_pred_class))
print(recall_score(y_test, lr_pred_class))

rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_pred_class = rfc.predict(X_test)
rfc_prob = rfc.predict_proba(X_test)
print('Random Forest')
print(accuracy_score(y_test, rfc_pred_class))
print(precision_score(y_test, rfc_pred_class))
print(recall_score(y_test, rfc_pred_class))
print('SVC')
svm = LinearSVC(class_weight = 'balanced', random_state = 27)
clf = CalibratedClassifierCV(svm)
clf.fit(X_train, y_train)
clf_pred = clf.predict(X_test)
clf_prob = clf.predict_proba(X_test)
print(accuracy_score(y_test, clf_pred))
print(precision_score(y_test, clf_pred))
print(recall_score(y_test, clf_pred))

Multinomial NB
0.7212681638044914
0.1315136476426799
0.424
Logistic Regression
0.7992073976221928
0.14901960784313725
0.304
Random Forest
0.8645970937912814
0.10784313725490197
0.088
SVC
0.8540290620871862
0.17567567567567569
0.208


In [11]:
mnb.coef_

array([[-9.57045886, -9.57045886, -9.57045886, ..., -8.59841678,
        -9.57045886, -9.57045886]])

In [12]:
def show_most_informative_features(vectorizer, mnb, n=20):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(mnb.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

In [13]:
show_most_informative_features(vectorizer, mnb, n=20)

	-9.5705	01             		-4.6061	thank          
	-9.5705	025            		-4.8772	think          
	-9.5705	065            		-4.9874	peopl          
	-9.5705	08             		-4.9939	presid         
	-9.5705	10             		-5.0539	let            
	-9.5705	10000          		-5.1048	work           
	-9.5705	100000         		-5.2271	make           
	-9.5705	1010           		-5.3021	unit           
	-9.5705	10th           		-5.3264	countri        
	-9.5705	11000          		-5.3582	want           
	-9.5705	112            		-5.4476	state          
	-9.5705	118000         		-5.4909	need           
	-9.5705	11th           		-5.4913	american       
	-9.5705	12             		-5.5189	america        
	-9.5705	12000          		-5.5967	like           
	-9.5705	1280           		-5.6106	yes            
	-9.5705	12th           		-5.6270	street         
	-9.5705	13             		-5.6283	know           
	-9.5705	1300           		-5.6446	wall           
	-9.5705	13000          		-5.6631	republican     


In [14]:
cheer = X[X.Label2 == 1]
not_cheer = X[X.Label2 == 0]

y_train = upsampled.Label2
X_train = upsampled.drop('Label2', axis=1)

svm = SVC(class_weight = 'balanced', random_state = 27)
clf = CalibratedClassifierCV(svm)
clf.fit(X_train, y_train)
clf_pred = clf.predict(X_test)
clf_prob = clf.predict_proba(X_test)
print(accuracy_score(y_test, clf_pred))
print(precision_score(y_test, clf_pred))
print(recall_score(y_test, clf_pred))

KeyboardInterrupt: 

In [ ]:
pres = pd.read_csv('Presidential')

In [ ]:
pres

In [ ]:
vector = document_vectorizer(primaries['tokenized'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vector, primaries['Labels2'])


In [ ]:
# nb = MultinomialNB()
# nb.fit(X_train, y_train)
# y_pred_class = nb.predict(X_test)
# y_prob = nb.predict_proba(X_test)
print(accuracy_score(y_test, y_pred_class))
print(precision_score(y_test, y_pred_class))
print(recall_score(y_test, y_pred_class))

lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred_class = lr.predict(X_test)
lr_prob = lr.predict_proba(X_test)

print(accuracy_score(y_test, lr_pred_class))
print(precision_score(y_test, lr_pred_class))
print(recall_score(y_test, lr_pred_class))

# rfc = RandomForestClassifier()
# rfc.fit(X_train, y_train)
# rfc_pred_class = rfc.predict(X_test)
# rfc_prob = rfc.predict_proba(X_test)

print(accuracy_score(y_test, rfc_pred_class))
print(precision_score(y_test, rfc_pred_class))
print(recall_score(y_test, rfc_pred_class))

svm = LinearSVC(class_weight = 'balanced', random_state = 27)
clf = CalibratedClassifierCV(svm)
clf.fit(X_train, y_train)
clf_pred = clf.predict(X_test)
clf_prob = clf.predict_proba(X_test)
print(accuracy_score(y_test, clf_pred))
print(precision_score(y_test, clf_pred))
print(recall_score(y_test, clf_pred))

In [ ]:
def calculate_threshold_values(prob, y):
    '''
    Build dataframe of the various confusion-matrix ratios by threshold
    from a list of predicted probabilities and actual y values
    '''
    df = pd.DataFrame({'prob': prob, 'y': y})
    df.sort_values('prob', inplace=True)
    
    actual_p = df.y.sum()
    actual_n = df.shape[0] - df.y.sum()

    df['tn'] = (df.y == 0).cumsum()
    df['fn'] = df.y.cumsum()
    df['fp'] = actual_n - df.tn
    df['tp'] = actual_p - df.fn

    df['fpr'] = df.fp/(df.fp + df.tn)
    df['tpr'] = df.tp/(df.tp + df.fn)
    df['precision'] = df.tp/(df.tp + df.fp)
    df = df.reset_index(drop=True)
    return df
    
def plot_roc(ax, df):
    ax.plot([1]+list(df.fpr), [1]+list(df.tpr), label="ROC")
    ax.plot([0,1],[0,1], 'k', label="random")
    ax.set_xlabel('fpr')
    ax.set_ylabel('tpr')
    ax.set_title('ROC Curve')
    ax.legend()
    
def plot_precision_recall(ax, df):
    ax.plot(df.tpr,df.precision, label='precision/recall')
    #ax.plot([0,1],[0,1], 'k')
    ax.set_xlabel('recall')
    ax.set_ylabel('precision')
    ax.set_title('Precision/Recall Curve')
    ax.plot([0,1],[df.precision[0],df.precision[0]], 'k', label='random')
    ax.set_xlim(xmin=0,xmax=1)
    ax.set_ylim(ymin=0,ymax=1)

In [ ]:
fpr = dict()
tpr = dict()

roc_auc = dict()

for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(y_test, y_prob[:,1])
    
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred_class.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

lpr = dict()
ipr = dict()

loc_auc = dict()

for i in range(2):
    lpr[i], ipr[i], _ = roc_curve(y_test, clf_prob[:,1])
    
lpr["micro"], ipr["micro"], _ = roc_curve(y_test.ravel(), clf_pred.ravel())
loc_auc["micro"] = auc(lpr["micro"], ipr["micro"])

rpr = dict()
dpr = dict()
doc_auc = dict()

for i in range(2):
    rpr[i], dpr[i], _ = roc_curve(y_test, rfc_prob[:,1])
    
rpr["micro"], dpr["micro"], _ = roc_curve(y_test.ravel(), rfc_pred_class.ravel())
doc_auc["micro"] = auc(rpr["micro"], dpr["micro"])

xpr = dict()
zpr = dict()
doc_auc = dict()

for i in range(2):
    xpr[i], zpr[i], _ = roc_curve(y_test, lr_prob[:,1])
    
xpr["micro"], zpr["micro"], _ = roc_curve(y_test.ravel(), lr_pred_class.ravel())
doc_auc["micro"] = auc(xpr["micro"], zpr["micro"])


plt.figure(figsize = (16,10))
lw = 5
plt.plot(fpr[1], tpr[1], color='darkorange',
         lw=lw, label='MultinomialNB')
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.plot(lpr[1], ipr[1], color='green',
        lw=5, label='Linear SVC')
plt.plot(rpr[1], dpr[1], color = 'red',
        lw=lw, label = 'Random Forest')
plt.plot(xpr[1], zpr[1], color = 'purple',
        lw=lw, label = 'Logistic Regression')
plt.xlim([0.0, 1.0])
plt.xticks(fontsize = 24)
plt.yticks(fontsize = 24)
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize = 36)
plt.ylabel('True Positive Rate', fontsize = 36)
plt.title('ROC Curves', fontsize = 48)
plt.legend(loc="lower right", fontsize= 36)
plt.show()

In [ ]:
calculate_threshold_values(lr_pred_class, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, lr_pred_class).ravel()

In [ ]:
tn, fp, fn, tp